# CV part one

В этой тетрадке мы рассмотрим задачу распознавания лиц на примере датасета [CelebA](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html)

**Предполагаем, что ноутбук запущен внутри Yandex DataSphere**

In [ ]:
# from pathlib import Path

import numpy as np
# import pandas as pd
# from tqdm import tqdm
# import cv2

# import torch
import torch.nn as nn
# import torch.nn.functional as F
# from torch import optim
from torch.utils.data import Dataset, DataLoader, Sampler
from torchvision.models import resnet34
# from torch.utils.tensorboard import SummaryWriter

## Data

Качаем архив с данными с Yandex Object Storage и распаковываем в текущую папку.

Структура архива:
- /celeba_data/
    - train.csv
    - val.csv
    - images/{image}.jpg

CSV файлы содержат название файла (`image`) и его лейбл (`label`).

In [ ]:
from cloud_ml.storage.api import Storage

s3 = Storage.s3(access_key="Le9tg70HQEJsoGqjqXH8", secret_key="PUT_SECRET_HERE")
# downloading contents of the remote file into the local one
s3.get('dl-hse-2021/celeba_data.zip', './celeba_data.zip')

In [ ]:
#!:bash
unzip -q ./celeba_data.zip -d ./ && rm celeba_data.zip

## Задание 1
**(0.2 балла)** Напишите класс датасет, который будет возвращать картинку и ее лейбл.

In [ ]:
class CelebADataset(Dataset):
    def __init__(self):
        pass

    def __len__(self):
        pass
    
    def __getitem__(self, index):
        pass

## Задание 2
**(0.2 балла)** Напишите функцию, которая будет считать метрику top-n accuracy.

$$TopN \ Accuracy = \frac{Number \ of \ objects \ with \ correct \ answer \ among \ topN \ predictions}{Total \ number \ of \ objects}$$

*Example:*

![image](https://www.baeldung.com/wp-content/ql-cache/quicklatex.com-ae746981c7a437b7e1fc2831e5d76d57_l3.svg)  
$Top3 \ Accuracy = \frac{4}{5} = 0.8$

*Hint:* Для каждого объекта выбираем `n` наиболее уверенных предсказаний. Если среди них есть правильный ответ, то увеличиваем числитель и знаменатель на единицу, иначе увеличиваем только знаменатель.

In [ ]:
def top_n_accuracy(preds: np.ndarray,
                   targets: np.ndarray,
                   n_size: int) -> float:
    pass

## Задание 3
**(0.2 балла)** Решите задачу без дообучения.

*Step-by-step:*
1. Инициализируйте предобученную сетку (`backbone`).
1. Прогоните через нее все картинки из валидационного датасета и сложите полученные эмбеддинги в массив.
1. Для каждого вектора найдите ближайшие к нему векторы и отсортируйте их по расстоянию. Лейблы соседних векторов будут предсказаниями для текущего вектора.
1. Оставьте топ-5 уникальных предсказаний.
1. Посчитайте и выведите метрики:
    1. top-1 accuracy
    1. top-5 accuracy

*Вопросы:*
1. Зачем мы заменяем последний линейный слой на `Identity` ?
1. Зачем используем на сетке метод `eval` ?

In [ ]:
backbone = resnet34(pretrained=True)
backbone.fc = nn.Identity()
backbone = backbone.eval()

## Задание 4
**(0.4 балла)** Решите задачу с дообучением на эмбеддингах.

*Step-by-step:*
1. Напишите небольшую сетку произвольной архитектуры, которая будет использовать эмбеды, выдаваемые `backbone` сетью.
1. Напишите класс Dataset, который будет возвращать эмбединг и лейбл.
1. Напишите класс Sampler [PyTroch docs](https://pytorch.org/docs/stable/data.html#data-loading-order-and-sampler), который будет отвечать за правильность сбора тренировочных батчей: якорный пример, позитивный, негативный.
1. Обучите ее на тренировочном датасете:
    1. Лосс -- [triplet loss](https://pytorch.org/docs/stable/generated/torch.nn.TripletMarginLoss.html).
    1. Метрика -- top-5 accuracy.
1. Посчитайте top-1 и top-5 accuracy на валидации. Насколько сильно они отличаются от того, что получилось в предыдущем задании?


*Hints:*
1. Убедитесь, что у каждого лейбла есть как минимум 2 примера, иначе не получится достать позитивный пример.
1. Лучше предварительно прогнать все картинки из трейна и сохранить полученные эмбеддинги, чтобы при обучении сети грузить только эмбеды (векторы).

In [ ]:
class CelebAEmbeddingDataset(Dataset):
    def __init__(self):
        pass

    def __len__(self):
        pass
    
    def __getitem__(self, index):
        pass
    

class CelebATripletSampler(Sampler):
    def __init__(self):
        pass

    def __len__(self):
        pass

    def __iter__(self):
        pass

In [ ]:
# your code must be before example

## Sampler (simple example)

В блоках ниже реализован пример датасета и сэмлера, который возвращает индексы для триплет лосса.

Датасет написан топорно, но основная логика следующая. Если ему на вход приходит `int`, то он возвращает название картинки (`img_name`) и ее лейбл (`img_label`). Если же приходит нечто длиной 3, то он возвращает 3 названия картинок, соответственно. В нашем случае это будет три картинки с двумя одинаковыми лейблами и одним другим: anchor, positive, negative.  
Сэмплер `SimpleTripletSampler`, в свою очередь, отвечает за формирование и поставку в датасет индексов триплетов.

Датасет и сэмлер объединяются внутри даталоадера.

*Hint:* Код написан только лишь для примера, поэтому логика возвращения триплетов может быть неверной.

In [ ]:
class SimpleDataset(Dataset):
    def __init__(self, img_names: np.ndarray,
                 img_labels: np.ndarray):
        if len(img_names) != len(img_labels):
            raise ValueError('img_names and img_labels must have equal number of elements')

        self.img_names = img_names
        self.img_labels = img_labels

    def __len__(self):
        return len(self.img_names)
    
    def __getitem__(self, idx):
        if isinstance(idx, int):
            img_name = self.img_names[idx]
            img_label = self.img_labels[idx]
            
            return img_name, img_label
        else:
            assert len(idx) == 3
            
            anc_idx, pos_idx, neg_idx = idx
            anc_img_name = self.img_names[anc_idx]
            pos_img_name = self.img_names[pos_idx]
            neg_img_name = self.img_names[neg_idx]

            return anc_img_name, pos_img_name, neg_img_name


class SimpleTripletSampler(Sampler):
    def __init__(self, dataset: Dataset):
        super().__init__(dataset)

        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __iter__(self):
        for anchor_idx in range(len(self.dataset)):
            positive_idx = self._mine_positive(anchor_idx)
            negative_idx = self._mine_negative(anchor_idx)

            yield anchor_idx, positive_idx, negative_idx

    def _mine_positive(self, anchor_idx: int):
        labels: np.ndarray = self.dataset.img_labels

        anchor_label = labels[anchor_idx]
        pos_idxs = np.nonzero(labels == anchor_label)[0]
        pos_idx = np.random.choice(pos_idxs)

        return pos_idx

    def _mine_negative(self, anchor_idx: int):
        labels: np.ndarray = self.dataset.img_labels

        anchor_label = labels[anchor_idx]
        neg_idxs = np.nonzero(labels != anchor_label)[0]
        neg_idx = np.random.choice(neg_idxs)

        return neg_idx

In [ ]:
ex_size = 100
np.random.seed(42)

# в нашем примере названием картинки будет выступать число от 0 до 99, а лейблом число от 0 до 4.
ex_dataset = SimpleDataset(img_names=np.arange(ex_size),
                           img_labels=np.random.randint(0, 5, size=ex_size))
ex_sampler = SimpleTripletSampler(dataset=ex_dataset)

ex_loader = DataLoader(dataset=ex_dataset, batch_size=10, sampler=ex_sampler)

In [ ]:
# В этой ячейке мы дергаем первый батч с названиями картинок и достаем их лейблы, 
#  чтобы проверить действительно ли у них одинаковые или разные лейблы.
# Для тренировки сети с триплет лоссом сами лейблы нам не нужны будут.
#  Главное чтобы триплеты картинок формировались правильно: anchor, positive, negative

ex_batch = next(iter(ex_loader))

ex_batch_anc_labels = ex_dataset.img_labels[ex_batch[0]]
ex_batch_pos_labels = ex_dataset.img_labels[ex_batch[1]]
ex_batch_neg_labels = ex_dataset.img_labels[ex_batch[2]]

In [ ]:
print('All anchor and positive labels are equal:', np.all(ex_batch_anc_labels == ex_batch_pos_labels))
print('Any of anchor and negative labels are equal:', np.any(ex_batch_anc_labels == ex_batch_neg_labels))